<a href="https://colab.research.google.com/github/jparkgit/NumPy/blob/master/Notebook_Pytorch_Derivative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 2. Pytorch

## Derivative


# 3. Derivatives

## 3.1. Derivatives

Determine the derivative of

$ y = 2x^3+x $   

at   $x=1$.


In [ ]:
x = torch.tensor(1., requires_grad=True) #every operation on them should be tracked

y = 2 * x ** 3 + x

y.backward() # calculate derivatives and stores them in the respective tensors' .grad

x.grad

tensor(7.)

$ \frac{dy(x)}{dx} = 6x^{2} + 1$  

$ \frac{dy(x=1)}{dx} = 6*1^{2} + 1$

In [ ]:
x = torch.tensor(1., requires_grad=True) #every operation on them should be tracked
y = 2 * x ** 3 + x
z = torch.exp(y)
z.backward()
x.grad

tensor(140.5988)

In [ ]:
print(x)
print('data:',x.data)
print('grad:',x.grad)



tensor(1., requires_grad=True)
data: tensor(1.)
grad: tensor(140.5988)


In [ ]:
print("is_leaf:",x.is_leaf)
print("requires_grad:",x.requires_grad)

is_leaf: True
requires_grad: True


In [ ]:
print('data:',y.data)
print('grad_fn:',y.grad_fn)
print('grad:',y.grad)


data: tensor(3.)
grad_fn: <AddBackward0 object at 0x7ff520953dd0>
grad: None


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print("is_leaf:",y.is_leaf)
print("requires_grad:",y.requires_grad)

is_leaf: False
requires_grad: True


### 2-4. Partial Derivates

#### 참고  
* `requires_grads=True`인 tensor는 tensor를 바로 .numpy() 로 변환 불가능  
* `.detach().numpy()` 로 변환 가능

Try to determine partial derivative  $u$ of the following function where $u=2$ and $v=1$: $ f=uv+(uv)^2$


In [ ]:
# Practice: Calculate the derivative of f = u * v + (u * v) ** 2 at u = 2, v = 1

# Type the code here
u = torch.tensor(2., requires_grad=True)
v = torch.tensor(1., requires_grad=True)
y = u*v + (u*v)**2

y.backward()
print(u.grad, v.grad)

tensor(5.) tensor(10.)


### Exercise 2:
You are given

$y=\exp(2x_1)+x_2^2$.

Calculate $\triangledown y = \left( \frac{\delta y}{\delta x_1},
\frac{\delta y}{\delta x_2} \right)$ at $x_1=1.0$ and $x_2=2.0$.

In [ ]:
x = torch.tensor([1.0, 2.0], requires_grad=True, dtype=torch.float32)
y=torch.exp(2*x[0])+x[1]**2
y.backward()
x.grad

tensor([14.7781,  4.0000])

In [ ]:
### See why the following is not working
x = torch.tensor([1.0, 2.0], requires_grad=True, dtype=torch.float32)
y=torch.math.exp(2*x[0])
y.backward()
x.grad

AttributeError: ignored

## 2.5 Chain rules amd gradients



## Example 1:

You are given $f(x_1, x_2)=(x_1+2x_2^3)^2$.

* Calculate grad $f$ at $(x_1, x_2)=(1,2)$.

Here, we are presenting two solutions. What is the difference?

In [ ]:
x = torch.tensor([1.0, 2.0], requires_grad=True)
f = (x[0]+2*x[1]**3)**2
f.backward()
x.grad

tensor([ 34., 816.])

In [ ]:
x = torch.tensor([[1.0, 2.0]], requires_grad=True)
f = (x[0,0]+2*x[0,1]**3)**2
f.backward()
x.grad

tensor([[ 34., 816.]])

### Example 2:

Let $f(x)=x_1^2+2x_2$ and $g(t)=\begin{pmatrix} \sin(t) \\ \cos(t)\end{pmatrix}.$

In [ ]:
import torch
t=torch.tensor([1.0], requires_grad=True)
g=torch.sin(t)**2+2* torch.cos(t)
g.backward()
t.grad

tensor([-0.7736])

In [ ]:
import torch
t=torch.tensor([1.0], requires_grad=True)
g=[torch.sin(t), torch.cos(t)]
f=g[0]**2 + 2*g[1]
f.backward()
print(t.grad)

tensor([-0.7736])


## 2.6$^*$ Jacobian (Second derivatives)



In [ ]:
import torch
def myftn(x, y):
    return 2 * torch.exp(x) + 3 * y

inputs = (torch.rand(1), torch.rand(1)) # arguments for the function
print(inputs)
torch.autograd.functional.jacobian(myftn, inputs)

(tensor([0.3463]), tensor([0.5210]))


(tensor([[2.8276]]), tensor([[3.]]))

## HW 1:

You are given $X$ and $Y$. Using the linear regression, you want to have the following model

$\widehat{Y}=\widehat{\beta}_0+\widehat{\beta}_1X$

by minimizing the following square loss

$g(\beta_0, \beta_1)=\sum\limits_{i=1}^{20}(y_i-(\beta_0+\beta_1 x_i))^2$.

Answer the following questions.

1. Calculate grad $g$ at $(\beta_0, \beta_1)=(1.0, 0.5)$ without using autograd.

2. Calculate grad $g$ at $(\beta_0, \beta_1)=(1.0, 0.5)$ using autograd.

3. Reshape $X$ into the shape of $[20,1]$ and name it as $XX$. Then, calculate grad $g$ at $(\beta_0, \beta_1)=(1.0, 0.5)$ using autograd. (Caution: your answer should be the same as in 2.)

4. Stack $X$ and $Y$ to make $MY\_data1$ of shape $[20,2]$. Then, calculate grad $g$ at $(\beta_0, \beta_1)=(1.0, 0.5)$ using autograd. (Caution: your answer should be the same as in 2.)

5. Concatenate $X$ and $Y$ to make $MY\_data2$ of shape $[20,2]$. (Use the reshape if necessary) Then, calculate grad $g$ at $(\beta_0, \beta_1)=(1.0, 0.5)$ using autograd. (Caution: your answer should be the same as in 2.)

In [ ]:
X=torch.tensor([-3.0000e+00, -2.7000e+00, -2.4000e+00, -2.1000e+00, -1.8000e+00,
        -1.5000e+00, -1.2000e+00, -9.0000e-01, -6.0000e-01, -3.0000e-01,
        -2.3842e-08,  3.0000e-01,  6.0000e-01,  9.0000e-01,  1.2000e+00,
         1.5000e+00,  1.8000e+00,  2.1000e+00,  2.4000e+00,  2.7000e+00])

In [ ]:
Y = torch.tensor([-7.1452, -5.4253, -5.1977, -3.6225, -3.8022, -4.4101, -4.6622, -3.1932,
        -1.7325, -1.8879, -1.0742, -0.2320,  1.8226,  1.5453, -1.5535,  0.8857,
         1.7537,  3.1607,  1.8912,  4.0895])

## HW 2:

You are given

$f(x)=x^3-3x^2+4$

Answer the following questions.

1. Draw the graph of $(x,f(x))$ for $x\in [-5, 5]$.
2. Draw the graph of $(x,f^{\prime}(x))$ for $x\in [-5, 5]$ using augograd in torch.


